In [17]:
import tensorflow as tf
import  tensorflow.keras.backend as K
import  tensorflow.keras as keras
from tensorflow.keras.layers import Input
#
from tensorflow.keras.models import Model
#
########################### fixed size stack
#https://favtutor.com/blogs/infix-to-postfix-conversion
#@tf.keras.utils.register_keras_serializable()
class infixToPostfix_AD(keras.layers.Layer):
      def __init__(self, **kwargs):
          #
          super(infixToPostfix_AD, self).__init__(**kwargs)
      #  
      # about get_config  see the following link:
      # https://towardsdatascience.com/how-to-write-a-custom-keras-model-so-that-it-can-be-deployed-for-serving-7d81ace4a1f8
      def get_config(self):
          config = super().get_config()
          # save constructor args
          #config['input_dim'] = self.input_dim
          # 
          return config
      #
      #@tf.function
      def call(self, inputs):
          def infixToPostfix_FUN_AD(inputs): 
              inputs = tf.squeeze(inputs)
              # input is tensor of strings, for example:
              # inputs = tf.constant([["( 1 OR 2 )  AND 3 OR 4"],["( 10 OR 20 ) AND 30 OR 40"], ["( 100 OR 200 ) AND 300 OR 400"]])
              #
              def remove_extra_spaces(a):
                  a = tf.strings.strip(a)
                  a = tf.strings.regex_replace(a, ' +', " ")
                  return a
              #
              def stack_not_empty(stack_top_index):
                  return tf.math.greater_equal(stack_top_index, 0)
              #
              def Original_push_stack(stack, update, number_of_rows, stack_top_index):
                  #  
                  print("push, number_of_rows: ", number_of_rows)
                  update = tf.reshape(update, tf.shape(stack)[0])
                  #
                  stack_top_index = tf.add(stack_top_index, 1)
                  #
                  rr      = tf.reshape(tf.range(tf.shape(stack)[0]), (number_of_rows, 1))
                  cc      = tf.repeat([[stack_top_index]], tf.shape(stack)[0], axis = 0)
                  indeces = tf.concat([rr, cc], axis = 1 )
                  #
                  return stack_top_index, tf.tensor_scatter_nd_update(stack, indeces, update) 
              #  
              def push_stack(stack, update, number_of_rows, stack_top_index):
                  #  
                  update = tf.squeeze(update)
                  #
                  #update = tf.reshape(update, (tf.size(update))  )
                  #
                  stack_top_index = tf.add(stack_top_index, 1)
                  #
                  rr      = tf.reshape(tf.range(tf.size(update)), (-1, 1))
                  cc      = tf.repeat([[stack_top_index]], number_of_rows, axis = 0)
                  indeces = tf.concat([rr, cc], axis = 1 )
                  #
                  return stack_top_index, tf.tensor_scatter_nd_update(stack, indeces, update) 
              #
              #
              def pop_stack(stack, number_of_rows, stack_top_index):
                  #
                  if tf.greater_equal(stack_top_index, 0):  
                     stack_2D_top = tf.slice(stack, [0, stack_top_index], [number_of_rows, 1])
                     stack_2D_top = tf.reshape(stack_2D_top, [number_of_rows, 1])
                     #
                     stack_top_index = tf.add(stack_top_index, -1)
                     #
                  else:
                     stack_2D_top = ""
                  return stack_top_index, stack_2D_top 
              #
              def output_update(outputs, stack_2D_top, number_of_rows):
                  #
                  outputs = tf.concat([outputs, stack_2D_top], axis = 1) 
                  outputs = tf.map_fn(lambda x: tf.strings.join([tf.gather(x, 0), " ", tf.gather(x, 1)]), outputs)  
                  #
                  outputs = tf.reshape(outputs, [number_of_rows, 1])
                  #
                  return outputs
              #
              def top_one_row_stack(stack, stack_top_index):
                  #
                  stack_row = tf.gather(stack, 0)
                  #
                  if stack_not_empty(stack_top_index):  
                     stack_top = tf.gather(stack_row, stack_top_index)
                  else:
                     stack_top = "ARBITRARY VALUE"
                  #
                  return stack_top
                
              # table has priority levels: {"OR": 1, "AND": 2, "NOT": 3}
              keys_tensor = tf.constant(['OR', 'AND', 'NOT'])
              vals_tensor = tf.constant([1, 2, 3])
              input_tensor = tf.constant(['NOT'])
              table = tf.lookup.StaticHashTable(
                      tf.lookup.KeyValueTensorInitializer(keys_tensor, vals_tensor),
                      default_value=-1)
              #
              # remove more than 1 spaces in a row and also beginning and ending spaces
              inputs = tf.map_fn(lambda x: remove_extra_spaces(x), inputs)
              #
              # number_of_columns should be 1 but number_of_rows s the number of rows i inputs  
              #
              row               = tf.gather(inputs, 0)
              number_of_columns = tf.size(row)
              number_of_rows    = tf.cast(tf.divide(tf.size(inputs), number_of_columns), dtype = tf.int32)
              number_of_rows = tf.size(inputs)  
              #
              n_components = tf.size(tf.strings.split([row]))
              #
              # split every row into tensor of words and result is, for example:
              #  tf.Tensor(
              # [['1'   'OR' '2'   'AND' '3'   'OR' '4'  ]
              #  ['10'  'OR' '20'  'AND' '30'  'OR' '40' ]
              #  ['100' 'OR' '200' 'AND' '300' 'OR' '400']], shape=(3, 7), dtype=string)
              splitted_inputs = tf.cast(tf.strings.split(inputs), dtype = tf.string).to_tensor()  
              splitted_inputs = tf.squeeze(splitted_inputs)  
              #
              # expression is the first row of the splitted_inouts, for example:
              # expression:  tf.Tensor(['1' 'OR' '2' 'AND' '3' 'OR' '4'], shape=(7,), dtype=string)
              # we use the first rwo to decide what to do for the entire splitted_inputs
              expression = tf.gather(splitted_inputs, 0)
              #
              #####################################################################################################################
              #
              n = number_of_rows * n_components
              stack   = tf.repeat([[""]], repeats=[n], axis=0)
              outputs = tf.repeat([[""]], repeats=[number_of_rows], axis=0)
              #
              stack   = tf.reshape(stack,   shape = [number_of_rows, n_components]) 
              outputs = tf.reshape(outputs, shape = [number_of_rows, 1]) 
              stack_top_index = -1
              #
              # For example, we get the following stack sfter initialization:
              # INITIAL STACK:    tf.Tensor(
              #                              [['EMPTY']
              #                               ['EMPTY']
              #                               ['EMPTY']], shape=(3, 1), dtype=string)
              # INITIAL OUTPUTs:  tf.Tensor(
              #                              [['EMPTY']
              #                               ['EMPTY']
              #                               ['EMPTY']], shape=(3, 1), dtype=string)
              ####################################################################################################################        
              #
              for i in tf.range(tf.size(expression)):
                  #
                  character = tf.gather(expression, i)
                  #
                  character_columns = tf.slice(splitted_inputs, [0, i], [number_of_rows, 1]) # get current column
                  #
                  #if character not in Operators:  # if an operand append in postfix expression
                  #
                  characterREG = character
                  if tf.equal(character, "(") or tf.equal(character, ")"):
                     # we add "\" to "(" or ")" because otherwise regex_full_match consider it as command not as a symbol
                     characterREG =  tf.strings.join(["\\", character]) 
                  #
                  isoperator = tf.math.reduce_any(tf.strings.regex_full_match(['OR', 'AND', 'NOT', '(', ')'],  characterREG))
                  #
                  # tf.math.logical_not(isoperator) is TRUE when we have the number not a command
                  if   tf.math.logical_not(isoperator): # not isoperator
                       #
                       # in this case we add current symbol for every row to outputs, 
                       # the previous code: output+= " " + character 
                       outputs = output_update(outputs, character_columns, number_of_rows)
                       #
                  elif tf.equal(character, '(' ):  # else Operators push onto stack
                       # the previos command was: stack.append('(')
                       # character_columns contains   
                       # push()
                       stack_top_index, stack = push_stack(stack, character_columns, number_of_rows, stack_top_index) 
                       #
                  elif tf.equal(character, ')' ):

                      #while stack and stack[-1]!= '(':
                      #
                      stack_top = top_one_row_stack(stack, stack_top_index)
                      #
                      while stack_not_empty(stack_top_index) and tf.not_equal(stack_top, "("):
                            #output+= " " + stack.pop()
                            #
                            stack_top_index, stack_2D_top = pop_stack(stack, number_of_rows, stack_top_index)
                            #
                            stack_top = top_one_row_stack(stack, stack_top_index)
                            #
                            # add saved top of the stack for every row to outputs
                            outputs = output_update(outputs, stack_2D_top, number_of_rows)
                            #
                      #
                      #stack.pop()  # remove "("
                      #
                      stack_top_index, stack_2D_top = pop_stack(stack, number_of_rows, stack_top_index)
                      # 
                  else: 
                      #
                      stack_top = top_one_row_stack(stack, stack_top_index)
                      #
                      #while stack and stack[-1]!='(' and Priority[character]<=Priority[stack[-1]]:
                      while stack_not_empty(stack_top_index) and \
                            tf.not_equal(stack_top, "(") and        \
                            tf.math.less_equal(table.lookup(character), table.lookup(stack_top)):    
                            #
                            #####  utput+= " " + stack.pop()
                            #
                            # we need stack_top here in order to get priority operation in Priority[stack[-1]]   
                            stack_top = top_one_row_stack(stack, stack_top_index)
                            #
                            stack_top_index, stack_2D_top = pop_stack(stack, number_of_rows, stack_top_index)
                            #
                            # add saved top of the stack for every row to outputs
                            outputs = output_update(outputs, stack_2D_top, number_of_rows) 
                            #   
                      #### stack.append(character)
                      stack_top_index, stack = push_stack(stack, character_columns, number_of_rows, stack_top_index)

              #while stack:
              # we add to uptput everything what is left in stack:
              while stack_not_empty(stack_top_index):    
                    #
                    #output+= " " + stack.pop()
                    #
                    stack_top_index, stack_2D_top = pop_stack(stack, number_of_rows, stack_top_index)
                    #
                    # we add top os the stack to output
                    outputs = output_update(outputs, stack_2D_top, number_of_rows)
                    #
              return outputs
          # call
          return infixToPostfix_FUN_AD(inputs)
#
class Precision_AD(keras.layers.Layer):
      def __init__(self, **kwargs):
          #
          super(Precision_AD, self).__init__(**kwargs)
          #
          self.InfToPost = infixToPostfix_AD()
      #  
      def get_config(self):
          config = super().get_config()
          # save constructor args
          #config['input_dim'] = self.input_dim
          # 
          return config
      #
      #@tf.function
      def call(self, inputs):
          #
          print("call precision: ", inputs)
          inputs = self.InfToPost(inputs)
          #
          #inputs = self.DH(inputs)
          #
          return inputs 
##############################################################################################################
inputs = tf.constant([["( 1 OR 2 )  AND 3 OR NOT ( 4 OR 5 )"],["( 10 OR 20 ) AND 30 OR NOT ( 40 OR 50 )"], ["( 100 OR 200 ) AND 300 OR NOT ( 400 OR 500 )"]])
print("inputs: ")
print(inputs)
outputs = infixToPostfix_AD()(inputs)
display(outputs)
print("Next NN type prediction:")
#
isTest = True
#
if isTest:
   input_layer  = Input(shape = (1), dtype= tf.string)
   #
   output_layer = Precision_AD()(input_layer)  
   #
   model = Model(inputs = input_layer, outputs = output_layer)
   #
   print("!!!!!!!!!!!!!! prediction:")
   pred = model.predict(inputs, batch_size = 150)
   print(pred)
else:
   model.save("infixToPostfix_AD")   

inputs: 
tf.Tensor(
[[b'( 1 OR 2 )  AND 3 OR NOT ( 4 OR 5 )']
 [b'( 10 OR 20 ) AND 30 OR NOT ( 40 OR 50 )']
 [b'( 100 OR 200 ) AND 300 OR NOT ( 400 OR 500 )']], shape=(3, 1), dtype=string)


<tf.Tensor: shape=(3, 1), dtype=string, numpy=
array([[b' 1 2 OR 3 AND 4 5 OR NOT OR'],
       [b' 10 20 OR 30 AND 40 50 OR NOT OR'],
       [b' 100 200 OR 300 AND 400 500 OR NOT OR']], dtype=object)>

Next NN type prediction:
call precision:  Tensor("Placeholder:0", shape=(None, 1), dtype=string)
!!!!!!!!!!!!!! prediction:
call precision:  Tensor("IteratorGetNext:0", shape=(None, 1), dtype=string)
1/1 [==============================] - 2s 2s/step
[[b' 1 2 OR 3 AND 4 5 OR NOT OR']
 [b' 10 20 OR 30 AND 40 50 OR NOT OR']
 [b' 100 200 OR 300 AND 400 500 OR NOT OR']]


In [ ]:
# select " ( " + cast(precison_1, string) + " AND " + cast(precison_1, string) + " ) " + " OR "

In [ ]:
# test string input for Custom Layer
class string_AD(keras.layers.Layer):
      def __init__(self, **kwargs):
          #
          super(string_AD, self).__init__(**kwargs)
      #  
      def get_config(self):
          config = super().get_config()
          # save constructor args
          #config['input_dim'] = self.input_dim
          # 
          return config
      #
      #@tf.function
      def call(self, inputs):
          output = tf.map_fn(lambda x: tf.strings.join([x, ' Alex Dolia TEST']), inputs)
          #output = inputs
          return output
#
#
inputs = tf.constant([["( 1 OR 2 )  AND 3 OR 4"],["( 10 OR 20 ) AND 30 OR 40"], ["( 100 OR 200 ) AND 300 OR 400"]])
#
input_layer  = Input(shape = (1), dtype= tf.string)
#
output_layer = string_AD()(input_layer)
#
model = Model(inputs = input_layer, outputs = output_layer)
#
pred = model.predict(inputs, batch_size = 150)

pred 